In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
import asyncio

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2022-04-12 15:32:58,404 | INFO     |: Logger demo_notebook.log initialized.


# Start/Stop the Simulator and Plotter

In [2]:
# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

In [3]:
# Reset the simulator
RESET_SIM()

# Start the plotter
RESET_PLOTTER()


## Commander class API
Units of measurement are meters and radians.

<table align="left">
    <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="font-family:monospace">Utility Functions</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">sim_is_running()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the run status of the simulator.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plotter_is_running()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the run status of the plotter.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="font-family:monospace">Plotter Functions</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_odom(x,y)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot a point (x,y) in the plotter in red. Units are (meters, meters).</span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_gt(x,y)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot a point (x,y) in the plotter in green. Units are (meters, meters).</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_bel(x,y)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot a point (x,y) in the plotter in blue. Units are (meters, meters).</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">plot_map()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Plot the map based on the map lines in <em>world.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reset_plotter()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reset the plots in the plotter.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="font-family:monospace">Simulator Functions</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">set_vel(linear_vel, angular_vel)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Set the linear velocity (m/s) and angular velocity (rad/s) of the virtual robot.</span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">get_pose()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the odometry and ground truth poses of the virtual robot as two numpy arrays. The units of each pose are (meters, meters, radians)</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">get_sensor()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Get the ToF sensor data (in meters) of the virtual robot as a numpy column array.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reset_sim()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reset the virtual robot to its initial pose.</span></th>
    </tr>    
   
</table>


## Open-Loop Control

In [4]:
# Reset Plotter and Simulator
cmdr.reset_plotter()
cmdr.reset_sim()

def plot_current_gt():
    odom, gt = cmdr.get_pose()
    cmdr.plot_odom(odom[0], odom[1])
    cmdr.plot_gt(gt[0], gt[1])

async def time_based_soln():
    """
    Uses a timer to decide when to change actions
    Will run differently for everyone because time
    depends on how fast this computer can process
    the changes (which is different from others)
    """
    for _ in range(4):
        plot_current_gt()
        cmdr.set_vel(0.5, 0)
        await asyncio.sleep(1)
        plot_current_gt()
        cmdr.set_vel(0, np.pi/2)
        await asyncio.sleep(1)
    cmdr.set_vel(0,0)

time1 = None
times = []
def get_vel_time():
    for i in range(100):
        time1 = time.time()
        cmdr.set_vel(0,np.pi/2 * (-1 if i % 2 == 1 else 1))
        times.append(time.time() - time1)
    print(f"Average time: {sum(times) / len(times)}")
    print(f"Std dev: {np.std(times)}")
    # 100 times, no math:
    # Average time: 0.010912902355194092
    # Std dev: 0.00784329388949928

    # 100 times, yes math:
    # Average time: 0.011290438175201416
    # Std dev: 0.007699923022536731

# Loop for pose
while cmdr.sim_is_running() and cmdr.plotter_is_running():
    await time_based_soln()
    
    

Loading Flatland...
Initializing pygame framework...


Exception: No valid data from Simulator; Simulator is probably not running!